# YouTube Trending Videos Statistical Analysis

## INTRODUCTION

#### to be edited</b>
<b>https://www.kaggle.com/rsrishav/youtube-trending-video-dataset

In [138]:
# Importing libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_1samp

%matplotlib inline

In [139]:
'''
Later on we can also remove the ones we wouldn't use
'''
# Importing dataset
data_gb = pd.read_csv("../data/GB_youtube_trending_data.csv",sep=",")  #uk  
data_fr = pd.read_csv("../data/FR_youtube_trending_data.csv",sep=",")  #france
data_us = pd.read_csv("../data/US_youtube_trending_data.csv",sep=",")  #usa 
data_ru = pd.read_csv("../data/RU_youtube_trending_data.csv",sep=",")  #russia
data_de = pd.read_csv("../data/DE_youtube_trending_data.csv",sep=",")  #germany 
data_ca = pd.read_csv("../data/CA_youtube_trending_data.csv",sep=",")  #canada   
data_kr = pd.read_csv("../data/KR_youtube_trending_data.csv",sep=",")  #southkorea   
data_jp = pd.read_csv("../data/JP_youtube_trending_data.csv",sep=",")  #japan
data_br = pd.read_csv("../data/BR_youtube_trending_data.csv",sep=",")  #brazil
data_mx = pd.read_csv("../data/MX_youtube_trending_data.csv",sep=",")  #mexico

In [140]:
'''
This can stay for now to easily see what are our properties in the dataset. We can delete it once
we are familiar with the dataset
'''
brief_list_col = []
brief_list = []

for i in range(0,len(data_gb.columns)):
    brief_list_col.append(data_gb.columns[i])
    brief_list.append(data_gb[data_gb.columns[i]][0])

brief_df = pd.DataFrame(brief_list_col,columns=["Property"])
brief_df.insert(1,"Output",brief_list,True)
brief_df

,Property,Output
0,video_id,J78aPJ3VyNs
1,title,I left youtube for a month and THIS is what ha...
2,publishedAt,2020-08-11T16:34:06Z
3,channelId,UCYzPXprvl5Y-Sf0g4vX-m6g
4,channelTitle,jacksepticeye
5,categoryId,24
6,trending_date,2020-08-12T00:00:00Z
7,tags,jacksepticeye|funny|funny meme|memes|jacksepti...
8,view_count,2038853
9,likes,353790


                                       **INDIVIDUAL PARTS**

In [141]:
gb_json = pd.read_json('../data/GB_category_id.json')

In [142]:
col_list = ["items","items_snippet"]

def flat_func(df,column):
    for column in col_list:
        flatten = pd.DataFrame(dict(df[column])).transpose()
        columns = [str(i) for i in flatten.columns]
        flatten.columns = [column + "_" + str(colname) for colname in columns]
        df = pd.concat([df,flatten],axis=1)
        df = df.drop(column,axis=1)
    return df

gb_flat = flat_func(gb_json,col_list)

In [143]:
'''
CategoryId 29 is missing in JSON file so I am going to remove those 3 items in order to run 
below code.
'''
drop_idx = data_gb[data_gb.categoryId == 29].index
data_gb = data_gb.drop(drop_idx)

trend = []
for i in data_gb["categoryId"]:
    for j in range(0,len(gb_json_trends)):
        if i == gb_json_trends["categoryId"][j]:
            trend.append(gb_json_trends["items_snippet_title"][j])

data_gb.insert(6,"trend",trend)

In [145]:
data_gb.sample(10)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trend,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
10676,jUBjKlsBwPE,Years Go By,2020-10-01T10:06:59Z,UCCnPBkdur_5DHdsLI2pQ1qw,Bryson Tiller - Topic,10,Music,2020-10-04T00:00:00Z,Bryson Tiller|ブライソンティラー|A N N I V E R S A R Y|...,315083,10344,110,472,https://i.ytimg.com/vi/jUBjKlsBwPE/default.jpg,False,False,Provided to YouTube by TrapSoul/RCA RecordsYea...
152,Wr41FbfaOWw,Britain Backtracking on Brexit Promises? Brita...,2020-08-08T08:30:01Z,UCSMqateX8OA2s1wsOR2EgJA,TLDR News,25,News & Politics,2020-08-12T00:00:00Z,[None],193647,5065,234,2061,https://i.ytimg.com/vi/Wr41FbfaOWw/default.jpg,False,False,Pick Up Your Copy: https://tldrnews.co.uk/prod...
9248,_AXx2XSI4Kw,Unboxing the Apple Mask,2020-09-24T13:05:37Z,UCsTcErHg8oDvUnTzoqsYeNw,Unbox Therapy,28,Science & Technology,2020-09-27T00:00:00Z,unbox therapy|unboxtherapy|tech|technology|gad...,6727949,159466,17058,24103,https://i.ytimg.com/vi/_AXx2XSI4Kw/default.jpg,False,False,Here it is.. The official Apple mask.Latercase...
5090,EBy8ciLn4UU,THREE LIONS ARE BACK! 🦁 Newbies Arrive & Firs...,2020-09-02T16:00:20Z,UCNT2e7Og56vm5_V-yJWvglA,England,17,Sports,2020-09-06T00:00:00Z,england|england player arrivals|england team|e...,861929,19214,306,1957,https://i.ytimg.com/vi/EBy8ciLn4UU/default.jpg,False,False,The Three Lions returned to St. George's Park ...
8629,SiSGxf8XL8E,SuperM - One (Monster & Infinity) (Live From T...,2020-09-24T04:48:21Z,UC4nO0-dDzKWcZ6OlzE_Dvng,SuperMVEVO,10,Music,2020-09-24T00:00:00Z,superm|super m|superm ellen|superm one|one mon...,497361,136156,438,8015,https://i.ytimg.com/vi/SiSGxf8XL8E/default.jpg,False,False,SuperM's new single “One (Monster and Infinity...
4170,u0DdDaTb42I,*WHAT TO WEAR WHEN YOU DON'T KNOW WHAT TO WEAR...,2020-08-26T20:00:43Z,UCa3wuly5TEbB5ETH8Ga-S0g,Kaci-Jay,26,Howto & Style,2020-09-01T00:00:00Z,kaci|kaci jay|kaci jay conder|kaci conder|dad ...,198463,11534,175,1362,https://i.ytimg.com/vi/u0DdDaTb42I/default.jpg,False,False,*WHAT TO WEAR WHEN YOU DON'T KNOW WHAT TO WEAR...
5841,fFcU6tBGRZQ,Testing 16 Strange Products • Burning Question...,2020-09-09T14:00:08Z,UCDRbNGFusqlXX4a5vwi9ouQ,Vat19,24,Entertainment,2020-09-10T00:00:00Z,vat19|vat 19|vat-19|burning questions|strange ...,386919,16973,433,2156,https://i.ytimg.com/vi/fFcU6tBGRZQ/default.jpg,False,False,The Vat19 Crew is back to answer over a dozen ...
8263,PZAWGmSwmws,KSI Couldn't Believe This...,2020-09-20T14:28:44Z,UCHCxVGuzTseVaEu-K9IntSw,S-X,10,Music,2020-09-22T00:00:00Z,[None],459909,48143,267,1357,https://i.ytimg.com/vi/PZAWGmSwmws/default.jpg,False,False,Stream Dangerous: https://sx.fanlink.to/sx_dan...
3615,vRXZj0DzXIA,BLACKPINK - 'Ice Cream (with Selena Gomez)' M/V,2020-08-28T04:00:11Z,UCOmHUn--16B90oW2L6FRR3A,BLACKPINK,10,Music,2020-08-30T00:00:00Z,YG Entertainment|YG|와이지|K-pop|BLACKPINK|블랙핑크|블...,118872753,10370477,733798,2569640,https://i.ytimg.com/vi/vRXZj0DzXIA/default.jpg,False,False,BLACKPINK - ‘Ice Cream (with Selena Gomez)’Com...
10437,yTPC4P77bcs,30 FALL OUTFIT IDEAS 🍂,2020-10-01T19:24:34Z,UCnQhwPVwcP-DnbUZtIMrupw,bestdressed,26,Howto & Style,2020-10-03T00:00:00Z,fall outfit ideas|fall outfits|winter outfits|...,888781,106587,694,5005,https://i.ytimg.com/vi/yTPC4P77bcs/default.jpg,False,False,format inspired by allegra shaw ➭ https://bit....


Possible hypothesis;

more popular the video more like/less dislike,
tbc